<a href="https://colab.research.google.com/github/sarthakkaushik/Diploma-Program-in-ML-and-AI/blob/main/DCGAN_On_MNIST_Dataset_%26_Gradient_Tape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GradientTape

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime

In [2]:
# For automatic differentiation
# Very useful when we want to have fine control over the differentiation

# Refer: https://www.tensorflow.org/api_docs/python/tf/GradientTape

tf.keras.backend.clear_session()  # For easy reset of notebook state.
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape() as t:
    t.watch(x)
    y = x**2

print(t.gradient(y, x).numpy())

6.0


In [3]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)     # Will compute to 6.0
d2y_dx2 = g.gradient(dy_dx, x)  # Will compute to 2.0

print(dy_dx)
print(d2y_dx2)

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


In [4]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
  z = y * y
dz_dx = g.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = g.gradient(y, x)  # 6.0



RuntimeError: ignored

In [6]:
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  y = x * x
  z = y * y
dz_dx = g.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = g.gradient(y, x)  # 6.0

print(dz_dx,dy_dx)


del g  # Drop the reference to the tape

tf.Tensor(108.0, shape=(), dtype=float32) tf.Tensor(6.0, shape=(), dtype=float32)


In [7]:
# gradient w.r.t multiple variables

x = tf.constant(3.0)
y = tf.constant(4.0)
with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  g.watch(y)
  z = y * y + x * x
dz_dx_dy = g.gradient(z, [x,y])  

print(x)
print(y)
print(dz_dx_dy)


tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=8.0>]


## Deep Convolutional GAN (DCGAN)

In [8]:
# Source: https://www.tensorflow.org/tutorials/generative/dcgan

tf.keras.backend.clear_session()  # For easy reset of notebook state.

import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

In [9]:
# load all of MNIST dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
print(train_images.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [11]:
# Standarize image data
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
print(train_images.shape)

# [-1, 1] normalization
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1] # 127.5 because it lies between 0 to 255 size of the image

(60000, 28, 28, 1)


In [12]:
# Convert train_images into a tf.data.Dataset 
# Always do this step because it make easy for tensorflow to process the data and additionally give us more functionality 
BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Batch and shuffle the data
# Refer: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices
'''
Creates a Dataset whose elements are slices of the given tensors. 
The given tensors are sliced along their first dimension.
'''

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print(train_dataset)

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>
